In [2]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import calendar

In [24]:
# La fonction renvoie sous forme de dataframe, le taux d'occupation et le débit pour chaque heure de la journée


def get_road_data_by_date(year, month, day, libelle, libelle_nd_amont, libelle_nd_aval):

    # API Url
    api_url = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/comptages-routiers-permanents/records"

    # Request parameters
    params = {
        "limit": 24,
        "order_by": "t_1h",
        "timezone": "CET",
        "refine": [
            f"t_1h:{year}/{month}/{day}",
            f"libelle_nd_amont:{libelle_nd_amont}",
            f"libelle_nd_aval:{libelle_nd_aval}",
            f"libelle:{libelle}"
        ]
    }

    # Get response
    response = requests.get(api_url, params=params)

    # Fetch data
    if response.status_code == 200:
        records = response.json().get("results", [])
        if records:
            df = pd.DataFrame(columns=["year", "month", "day", "hour", "flow rate", "occupancy rate", "trafic state"])
            for index, record in enumerate(records):
                t_1h = record["t_1h"]
                q = record["q"]
                k = record["k"]
                trafic_state = record["etat_trafic"]
                year = t_1h[:4]
                month = t_1h[5:7]
                day = t_1h[8:10]
                hour = t_1h[11:13]
                
                df.loc[index] = [year, month, day, hour, q, k, trafic_state]
        else:
            print(f"No data found ({year}/{month}/{day}).")
            df = pd.DataFrame(columns=["year", "month", "day", "hour", "flow rate", "occupancy rate", "trafic state"])
    else:
        print(f"API Error : {response.status_code}")
    
    empty_df = pd.DataFrame({"year": year, "month": month, "day": day, "hour": [str(i).zfill(2) for i in range(24)]})

    filled_df = pd.merge(df, empty_df, on=['year', 'month', 'day', 'hour'], how="outer")
    return filled_df


In [26]:
print(get_road_data_by_date("2024", "11", "15", "AV_Champs_Elysees", "Av_Champs_Elysees-Washington", "Av_Champs_Elysees-Berri"))
#print(get_road_data_by_date("2023", "12", "06", "Convention", "Convention-Blomet", "Lecourbe-Convention"))
#print(get_road_data_by_date("2023", "12", "06", "St_Antoine", "Bastille-St_Antoine", "St_Antoine-Jacques_Coeur"))

# q: h flow rate (number of vehicle per hour)
# k: occupancy rate (time spent by a vehicle in one hour, in %, 100% = 1 hour)

    year month day hour  flow rate  occupancy rate trafic state
0   2024    11  15   00        NaN             NaN          NaN
1   2024    11  15   01        NaN             NaN          NaN
2   2024    11  15   02        NaN             NaN          NaN
3   2024    11  15   03        NaN             NaN          NaN
4   2024    11  15   04        NaN             NaN          NaN
5   2024    11  15   05        NaN             NaN          NaN
6   2024    11  15   06        NaN             NaN          NaN
7   2024    11  15   07        NaN             NaN          NaN
8   2024    11  15   08        NaN             NaN          NaN
9   2024    11  15   09        NaN             NaN          NaN
10  2024    11  15   10      254.0         4.09445       Fluide
11  2024    11  15   11        NaN             NaN          NaN
12  2024    11  15   12        NaN             NaN          NaN
13  2024    11  15   13        NaN             NaN          NaN
14  2024    11  15   14        NaN      

In [22]:
# La fonction renvoie sous forme de dataframe, le taux d'occupation et le débit, pour un mois donné


def get_road_data_per_month(year, month, libelle, libelle_nd_amont, libelle_nd_aval):

    number_of_days = calendar.monthrange(int(year), int(month))[1]

    dfs = []

    for index in range(number_of_days):
        day = str(index + 1).zfill(2)
        dfs.append(get_road_data_by_date(year, month, day, libelle, libelle_nd_amont, libelle_nd_aval))
    
    df = pd.concat(dfs, ignore_index=True)
    return df


In [25]:
print(get_road_data_per_month("2023", "11", "AV_Champs_Elysees", "Av_Champs_Elysees-Washington", "Av_Champs_Elysees-Berri"))
#print(get_road_data_per_month("2024", "11", "Convention", "Convention-Blomet", "Lecourbe-Convention"))
#print(get_road_data_per_month("2024", "11", "St_Antoine", "Bastille-St_Antoine", "St_Antoine-Jacques_Coeur"))

/var/folders/cp/rnb5h_ld0pzb64z4cgkyxd3h0000gn/T/ipykernel_18597/3594335461.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[index] = [year, month, day, hour, q, k, trafic_state]
/var/folders/cp/rnb5h_ld0pzb64z4cgkyxd3h0000gn/T/ipykernel_18597/3594335461.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[index] = [year, month, day, hour, q, k, trafic_state]
/var/folders/cp/rnb5h_ld0pzb64z4cgkyxd3h0000gn/T/ipykernel_18597/3594335461.py:40: FutureWarning: The behavior of DataFr

     year month day hour  flow rate  occupancy rate trafic state
0    2023    11  01   00        NaN             NaN          NaN
1    2023    11  01   01        NaN             NaN          NaN
2    2023    11  01   02        NaN             NaN          NaN
3    2023    11  01   03        NaN             NaN          NaN
4    2023    11  01   04      704.0        11.40389       Fluide
..    ...   ...  ..  ...        ...             ...          ...
715  2023    11  30   19     1457.0        28.16000   Pré-saturé
716  2023    11  30   20     1433.0        30.87000       Saturé
717  2023    11  30   21     1253.0        24.38111   Pré-saturé
718  2023    11  30   22     1069.0        20.40723   Pré-saturé
719  2023    11  30   23      950.0        16.66334   Pré-saturé

[720 rows x 7 columns]
